In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/gender_submission.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/train.csv


In [2]:
train_data = pd.read_csv("/kaggle/input/titanic/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
test_data = pd.read_csv("/kaggle/input/titanic/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [5]:
dfs = [train_data ,test_data]

In [6]:
for df in dfs:
    df['Cabin'].fillna(0, inplace = True)

In [7]:
cabins = []
for i in train_data['Cabin']:
    cabins.append(str(i))

In [8]:
letters = []
for i in cabins:
    letter= i[0]
    letters.append(letter)

In [9]:
train_data['Cabin'] = letters

In [10]:
cabins = []
for i in test_data['Cabin']:
    cabins.append(str(i))
letters = []
for i in cabins:
    letter= i[0]
    letters.append(letter)
test_data['Cabin'] = letters

In [11]:
train_data['Cabin'].head()

0    0
1    C
2    0
3    C
4    0
Name: Cabin, dtype: object

In [12]:
train_data['Embarked'].value_counts()

S    644
C    168
Q     77
Name: Embarked, dtype: int64

In [13]:
for df in dfs:
    df['Embarked'].fillna('S')

In [14]:
train_data['Family'] = train_data.apply(lambda x: x['SibSp'] + x['Parch'], axis = 1)
test_data['Family'] = test_data.apply(lambda x: x['SibSp'] + x['Parch'], axis = 1)

In [15]:
train_data.drop(['SibSp', 'Parch', 'Name', 'Ticket'], axis = 1, inplace = True)
test_data.drop(['SibSp', 'Parch', 'Name', 'Ticket'], axis = 1, inplace = True)

In [16]:
test_data.isna().sum()

PassengerId     0
Pclass          0
Sex             0
Age            86
Fare            1
Cabin           0
Embarked        0
Family          0
dtype: int64

In [17]:
test_data['Fare'].fillna(test_data['Fare'].median(), inplace = True)

In [18]:
for df in dfs:
    df['Age'].fillna(df['Age'].median(), inplace = True)

In [19]:
train_df = pd.get_dummies(train_data)
test_df = pd.get_dummies(test_data)

In [20]:
train_df.drop('PassengerId', axis = 1, inplace = True)

In [21]:
y = train_df["Survived"]
train_df.drop('Survived', axis = 1, inplace = True)
train_df.drop('Cabin_T', axis = 1, inplace = True)
test_df.drop('PassengerId', axis = 1, inplace = True)

In [22]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
X_test = test_df
X_train = train_df

In [23]:
rfc = RandomForestClassifier()

In [24]:
param_grid = {
    'n_estimators': [200, 500, 1000],
    'max_features': ['auto'],
    'max_depth': [6, 7, 8],
    'criterion': ['entropy']
}

In [25]:
CV = GridSearchCV(estimator = rfc, param_grid = param_grid, cv = 5)
CV.fit(X_train, y)
CV.best_estimator_

RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=1000)

In [26]:
rfc = RandomForestClassifier(criterion = 'entropy', max_depth = 8, n_estimators = 800)
rfc.fit(X_train, y)

RandomForestClassifier(criterion='entropy', max_depth=8, n_estimators=800)

In [27]:
y_pred = rfc.predict(X_test)
y_pred

array([0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [28]:
submission = y_pred.reshape(-1, 1)

In [29]:
sub_df = pd.DataFrame(submission)

In [30]:
sub_df['PassengerId'] = test_data['PassengerId']
sub_df['Survived'] = submission
cols = ['PassengerId',
       'Survived']
sub_df.drop(0, axis = 1, inplace = True)
sub_df.columns = [i for i in cols]
sub_df = sub_df.set_index('PassengerId')
sub_df.head(10)

,Survived
PassengerId,
892,0
893,0
894,0
895,0
896,1
897,0
898,0
899,0
900,1


In [31]:
sub_df.to_csv(r'submission.csv')